# Libraries

In [3]:
import pandas as pd
import numpy as np
import requests
import re

from bs4 import BeautifulSoup
from datetime import datetime

# 1. Project Plan

In this section, we created the plan to answer the questions asked by the partners. The questions were answered using *SAPE* method. *SAPE* is the method created by a Brazilian Data Scientist (Meigarom Lopes) in order to better organize the strategies to solve the business problems. The most appropriate translation of *SAPE* into English is *OPI*: output, process and input. 

#### 1. What is the best jeans sale price?

Output

1. How to answer the question.

- Median of the products from the competitors website.

2. Format

- Table or chart.

3. Mean of delivery.

- Streamlit App.

Process

1. Steps to calculate the answer.

- Price median per category, type and color.

2. What we will use to create the table and the chart.

- Simulation using Google Sheets.

3. How the final product will be.

- A dashboard with a Streamlit App. It will be published to Heroku (cloud environment).

Input

1. H&M: https://www2.hm.com/en_us/men/products/jeans.html

2. Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

#### 2. How many different types of jeans and colors should we choose?

This question must be answered by the previous plan. 

#### 3. What raw materials should we choose to make the jeans?  

We may answer this question by the same procedure chosen in the previous plan and selecting the composition on the websites. 

# 2. Data Collection 

In this section, we collected the attributes - defined in the previous section - from the Star Jeans competitors website: **H&M** and **Macys**.

## 2.1. H&M

### 2.1.1. Collection of id, product name, product type/category, price and datetime. 

The attributes collected in this section are part of the showcase for men jeans from H&M website page - but datetime. 

In [2]:
def get_webpage(url01, headers):
    page = requests.get(url01, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    return soup
    
def collect_data(soup):    
    products = soup.find('ul', 'products-listing small')
    
    product_id_category = products.find_all('article', 'hm-product-item')
    product_name = products.find_all('a', 'link')
    product_price = products.find_all('span', 'price regular')
    
    return product_id_category, product_name, product_price

def get_id(product_id_category):
    product_id = [p.get('data-articlecode') for p in product_id_category]
    
    return product_id

def get_category(product_id_category):
    product_category = [p.get('data-category') for p in product_id_category]
    
    return product_category

def get_name(product_name):
    product_name = [p.get_text() for p in product_name]
    
    return product_name

def get_price(product_price):
    product_price = [p.get_text() for p in product_price]

    return product_price

def create_dataframe(product_id, product_name, product_category, product_price):
    data = pd.DataFrame([product_id, product_name, product_category, product_price]).T
    data.columns = ['id', 'product_name', 'product_type', 'price']
    
    return data

def set_datetime():
    data['datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return data

if __name__ == '__main__':
    url01 = "https://www2.hm.com/en_us/men/products/jeans.html?sort=stock&image-size=small&image=model&offset=0&page-size=72"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    soup = get_webpage(url01, headers)
    
    product_id_category, product_name, product_price = collect_data(soup)
    
    product_id = get_id(product_id_category)
    
    product_category = get_category(product_id_category)
    
    product_name = get_name(product_name)
    
    product_price = get_price(product_price)
    
    data = create_dataframe(product_id, product_name, product_category, product_price)
    
    data = set_datetime()

In [3]:
data

,id,product_name,product_type,price,datetime
0,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16
1,1008549001,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16
2,0979945001,Loose Jeans,men_jeans_loose,$ 29.99,2022-01-27 19:43:16
3,0875105018,Relaxed Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 19:43:16
4,0811993040,Regular Jeans,men_jeans_regular,$ 29.99,2022-01-27 19:43:16
...,...,...,...,...,...
63,0974202002,Regular Denim Joggers,men_jeans_loose,$ 29.99,2022-01-27 19:43:16
64,0811993039,Regular Jeans,men_jeans_regular,$ 29.99,2022-01-27 19:43:16
65,0985197004,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-27 19:43:16
66,0993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-27 19:43:16


### 2.1.2. Collection of color, fit, composition, more sustainable materials and size

To collect the attributes of this section, we may not scrape data from the showcase because each product has its own specificity. As a solution, we selected each product and collected its attributes. 

#### 2.1.2.1. Pagination number

We must calculate the pagination number in order to achieve the number of pages that matches the number of products which are available in the showcase. 

In [4]:
def number_products(soup):
    total_itens = soup.find_all('h2', 'load-more-heading')[0].get('data-total')
    print(f"The total number of products is: {total_itens}.")
    
    return total_itens

def pagination(total_itens):
    products_per_page = 36
    pagination_number = np.round(int(total_itens)/products_per_page)
    print(f"The pagination number is: {pagination_number}.")
    
    return pagination_number

if __name__ == '__main__':
    total_itens = number_products(soup)
    
    pagination_number = pagination(total_itens)

The total number of products is: 68.
The pagination number is: 2.0.


#### 2.1.2.2. Available product attributes on website

This section is to certify what attributes are available for the products on the H&M website. 

In [5]:
def attributes_evaluation():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    # Unique columns for all products
    aux = []

    for i in range(len(data)):

        #API request
        # conteúdo de headers é padrão
        url = "https://www2.hm.com/en_us/productpage." + data.loc[i, 'id'] + ".html"

        page = requests.get(url, headers=headers)

        #Beautiful Soup object
        soup = BeautifulSoup(page.text, 'html.parser')

        # ============================= Color =========================

        #product list
        product_list = soup.find_all('a', 'filter-option miniature')

        #color
        product_color = [p.get('data-color') for p in product_list] 

        #id
        product_id = [p.get('data-articlecode') for p in product_list]

        #dataframe
        df_color = pd.DataFrame([product_id, product_color]).T
        df_color.columns = ['id', 'color']

        #generate style id + color id
        df_color['style_id'] = df_color['id'].apply(lambda x: x[:-3])
        df_color['color_id'] = df_color['id'].apply(lambda x: x[-3:])

        # ============================ Composition =====================

        # Product list
        product_composition_list = soup.find_all('div', 'pdp-description-list-item')

        # Composition
        product_composition = [list( filter( None, p.get_text().split('\n') ) ) for p in product_composition_list]


        # dataframe
        df_composition = pd.DataFrame(product_composition).T

        # Columns name

        df_composition.columns = df_composition.iloc[0]

        # Filling None/NA values
        df_composition = df_composition.iloc[1:].fillna(method='ffill')

        # ========== Columns we want
        aux = aux + df_composition.columns.tolist()

    return print(set(aux))

In [6]:
attributes_evaluation()

{'More sustainable materials', 'Art. No.', 'Size', 'Composition', 'Fit'}


#### 2.1.2.3 Data collection

In this section, we collected the data based on the attributes found in previous section. 

In [7]:
def data_collection():

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    #empty dataframe
    df_final = pd.DataFrame()

    # All columns found on website
    cols = ['Art. No.', 'Composition', 'Fit', 'More sustainable materials', 'Size']
    df_pattern = pd.DataFrame(columns=cols)

    for i in range(len(data)):

        #API request
        # conteúdo de headers é padrão
        url02 = "https://www2.hm.com/en_us/productpage." + data.loc[i, 'id'] + ".html"+ "?page-size=" + str(int(pagination_number*36))

        page = requests.get(url02, headers=headers)

        #Beautiful Soup object
        soup = BeautifulSoup(page.text, 'html.parser')

        # ============================= Color =========================

        #product list
        product_list = soup.find_all('a', role='radio')

        #color
        product_color = [p.get('data-color') for p in product_list] 

        #id
        product_id = [p.get('data-articlecode') for p in product_list]

        #dataframe
        df_color = pd.DataFrame([product_id, product_color]).T
        df_color.columns = ['id', 'color']

        #generate style id + color id
        df_color['style_id'] = df_color['id'].apply(lambda x: x[:-3])
        df_color['color_id'] = df_color['id'].apply(lambda x: x[-3:])

         # ============================ Composition =====================

        # Product list
        product_composition_list = soup.find_all('div', 'pdp-description-list-item')

        # Composition
        product_composition = [list( filter( None, p.get_text().split('\n') ) ) for p in product_composition_list]


        # dataframe
        df_composition = pd.DataFrame(product_composition).T

        # Columns name
        df_composition.columns = df_composition.iloc[0]

        # Filling None/NA values
        df_composition = df_composition.iloc[1:].fillna(method='ffill')

        # The same number of columns (pattern)
        df_composition = pd.concat( [df_pattern, df_composition] )

        # Generate Style ID + Color ID
        # All values, but the last three values
        df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
        df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

        # ======================= Merging color + composition ==========================
        data_merge = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')

        # ======================= Concatenate ==========================================
        df_final = pd.concat( [df_final, data_merge], axis=0 )
        
    return df_final

def data_merge(data, df_final):
    # Creating style_id + color_id
    data['style_id'] = data['id'].apply(lambda x: x[:-3])
    data['color_id'] = data['id'].apply(lambda x: x[-3:])

    data_raw = pd.merge( data, df_final[['color', 'style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')
    
    return data_raw

def save_csv(data_raw):
    data_raw.to_csv("data_raw.csv")
    
    return None
    
if __name__ == '__main__':
    df_final = data_collection()
    
    data_raw = data_merge(data, df_final)
    
    save_csv(data_raw)

In [8]:
data_merge(data, df_final)

,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size
0,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16,1008549,004,Denim blue,Regular fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
1,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16,1008549,004,Denim blue,Regular fit,"Pocket lining: Polyester 63%, Cotton 37%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
2,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16,1008549,004,Dark blue,Regular fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
3,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16,1008549,004,Dark blue,Regular fit,"Pocket lining: Polyester 63%, Cotton 37%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
4,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 19:43:16,1008549,004,Black,Regular fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
...,...,...,...,...,...,...,...,...,...,...,...,...
3869,0865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 19:43:16,0865734,001,Light denim blue,Relaxed fit,Cotton 100%,NaN,NaN
3870,0865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 19:43:16,0865734,001,Denim blue,Relaxed fit,Cotton 100%,NaN,NaN
3871,0865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 19:43:16,0865734,001,Light denim blue,Relaxed fit,Cotton 100%,NaN,NaN
3872,0865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 19:43:16,0865734,001,Gray,Relaxed fit,Cotton 100%,NaN,NaN


# 3. Data Cleaning

- We chose to drop the columns 'More sustainable materials' and 'Size' because they have a significative amount of NA values and they are not necessary for the analysis - according to our planning. 

- Product name content was put into lowercase and separated by underscore.

- At the column price, we removed the dollar sign and converted the values into float.

- We converted the date into datetime.

- We separated the color and the fit names with underscore and set them to lowercase.

- We removed the rows which contained the terms: 'Pocket', 'Pocket lining', 'Lining' e 'Shell'. The choice was made at this stage we do not need these type of materials. 

- We separated the materials into cotton, polyester and spandex.

- We created a dataframe (named 'df_ref') with the same number of rows that the dataframe named 'data' in order to concatenate them. 

In [21]:
data = pd.read_csv("https://github.com/lucasquemelli/Star_Jeans/blob/main/H%26M%20-%20webscraping/data_raw.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 133, saw 2


In [20]:
def show_data(data):
    print(data.head())
    
    return None

def data_types(data):
    print(f"\n{data.dtypes}")
    
    return None

def na_values(data):
    print(f"\nNumber of NA values for each column: {data.isna().sum()}")
    
    return None

def data_drop(data):
    data = data.drop([['More sustainable materials', 'Size']], axis=1)
    
    return data

def product_name(data):
    data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower() )
    
    return data

def product_price(data):
    data['price'] = data['price'].apply(lambda x: x.replace('$ ', '')).astype(float)
    
    return data

def conversion_date(data):
    data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S')
    
    return data

def show_colors(data):
    print(f"\nUnique colors are: {data['color'].unique()}")
    
    return None

def product_color(data):
    data['color'] = data['color'].apply(lambda x: x.replace(' ', '_').replace('/', '_').replace('-', '_').lower() )
    
    return data

def product_fit(data):
    data['Fit'] = data['Fit'].apply(lambda x: x.replace(' ', '_').lower())
    
    return data

def show_composition(data):
    print(f"\nThe compositions are: {data['Composition'].unique()}")

    return None

def remove_materials(data):
    data = data[~data['Composition'].str.contains('Pocket lining:')]
    data = data[~data['Composition'].str.contains('Lining:')]
    data = data[~data['Composition'].str.contains('Shell:')]
    data = data[~data['Composition'].str.contains('Pocket:')]

    return data

def composition_content(data):
    df1 = data['Composition'].str.split(',', expand=True)
    
    print("\ndf1[0].unique()")
    print("\ndf1[1].unique()")
    print("\ndf1[2].unique()")
    
    return df1
    
def create_dataframe(data):
    df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['Cotton', 'Polyester', 'Spandex'])
    
    return df_ref

def show_shape(data, df_ref):
    print(f"\nThe shape of the 'data' dataframe is: {data.shape}.")
    print(f"\nThe shape of the 'df_ref' dataframe is: {df_ref.shape}.")
    
    return None
    
def cotton_column(df1, df_ref):
    df_cotton = df1[0]
    df_cotton.name = 'cotton'

    df_ref = pd.concat([df_ref.reset_index(), df_cotton.reset_index()], axis=1)
    df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]
    df_ref = df_ref.drop(columns=['Cotton'], axis=1)
    
    return df_ref

def polyester_column(df1, df_ref):
    df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
    df_polyester.name = 'polyester'
    
    df_ref = pd.concat([df_ref.reset_index(), df_polyester.reset_index()], axis=1)
    df_ref = df_ref.drop(columns=['Polyester'], axis=1)
    
    df_ref = df_ref.drop(columns=['level_0'], axis=1)
    df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]
    
    return df_ref

def spandex_column(df1, df_ref):
    df_spandex = df1.loc[df1[1].str.contains('Spandex', na=True), 1]
    df_spandex.name = 'spandex'
    
    df_ref = pd.concat([df_ref.reset_index(), df_spandex.reset_index()], axis=1)
    df_ref = df_ref.drop(columns=['Spandex'], axis=1)
    
    df_ref = df_ref.drop(columns=['level_0'], axis=1)
    df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]
    
    return df_ref
    
def data_sample(df_ref):
    print(df_ref.sample(8))
    
    return None

def new_data(data, df_ref):
    data = pd.concat([data.reset_index(), df_ref.reset_index()], axis=1)
    data = data.drop(columns=['index', 'Unnamed: 0', 'level_0'], axis=1)
    data = data.iloc[:, ~data.columns.duplicated()]
    
    return data
    
def new_compostion(data):
    data['cotton'] = data['cotton'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
    data['polyester'] = data['polyester'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
    data['spandex'] = data['spandex'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
    
    return data
    
if __name__ == '__main__':    
    show_data(data)
    
    data_types(data)
    
    na_values(data)
    
    data = data_drop(data)
    
    data = product_name(data)
    
    data = product_price(data)
    
    data = conversion_date(data)
    
    show_colors(data)
    
    data = product_color(data)
    
    data = product_fit(data)
    
    show_composition(data)
    
    data = remove_materials(data)
    
    df1 = composition_content(data)
    
    df_ref = create_dataframe(data)
    
    show_shape(data, df_ref)
    
    df_ref = cotton_column(df1, df_ref)
                           
    df_ref = polyester_column(df1, df_ref)
    
    df_ref = spandex_column(df1, df_ref)
    
    data_sample(df_ref)
    
    data = new_data(data, df_ref)
    
    data = new_compostion(data)

C:\Users\USURIO~3\AppData\Local\Temp/ipykernel_1748/4212878889.py:145: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  data = get_data(url)


                                     <!DOCTYPE html>
0  <html lang="en" data-color-mode="auto" data-li...
1                                             <head>
2                             <meta charset="utf-8">
3    <link rel="dns-prefetch" href="https://githu...
4    <link rel="dns-prefetch" href="https://avata...

<!DOCTYPE html>    object
dtype: object

Number of NA values for each column: <!DOCTYPE html>    0
dtype: int64


b'Skipping line 133: expected 1 fields, saw 2\nSkipping line 152: expected 1 fields, saw 376\nSkipping line 153: expected 1 fields, saw 2\nSkipping line 168: expected 1 fields, saw 2\nSkipping line 221: expected 1 fields, saw 3\nSkipping line 235: expected 1 fields, saw 6\nSkipping line 259: expected 1 fields, saw 4\nSkipping line 264: expected 1 fields, saw 10\nSkipping line 269: expected 1 fields, saw 10\nSkipping line 274: expected 1 fields, saw 10\nSkipping line 279: expected 1 fields, saw 10\nSkipping line 284: expected 1 fields, saw 10\nSkipping line 289: expected 1 fields, saw 10\nSkipping line 294: expected 1 fields, saw 10\nSkipping line 299: expected 1 fields, saw 10\nSkipping line 304: expected 1 fields, saw 10\nSkipping line 309: expected 1 fields, saw 10\nSkipping line 314: expected 1 fields, saw 10\nSkipping line 325: expected 1 fields, saw 8\nSkipping line 329: expected 1 fields, saw 8\nSkipping line 337: expected 1 fields, saw 4\nSkipping line 342: expected 1 fields, sa

KeyError: "[('More sustainable materials', 'Size')] not found in axis"